In [115]:
#Motion file
from numpy import genfromtxt

motionRawData = genfromtxt(r'C:\Users\OndrejSpetko\Desktop\School\MED7\HRV-tracker\Breathing\motiondetection\motiondetection\out.txt', delimiter=',')
motionRawData.shape[0]

2900

In [116]:
#Sensor file
FSR_IR = genfromtxt(r'C:\Users\OndrejSpetko\Desktop\School\MED7\HRV-tracker\Breathing\SerialListener\Data\1.txt', delimiter=',')
FSR_IR.shape[0]

1232

In [117]:
#Functions
def scale(a):
    scalar = (np.mean(a, axis=0) - np.mean(motionData, axis=0))/np.mean(motionData, axis=0)
    #print(scalar)
    out = np.array(a, dtype=float)/scalar*30
    return out

def move(a):
    out = a
    out -= min(out) 
    return out

In [118]:
#Match sizes
import numpy as np
import math

while motionRawData.shape[0] > FSR_IR.shape[0] :
    every = math.ceil(motionRawData.shape[0]/(motionRawData.shape[0] - FSR_IR.shape[0]))
    motionRawData = np.delete(motionRawData, list(range(0, motionRawData.shape[0], every)), axis=0)
motionRawData.shape

(1232, 2)

In [119]:
#Time data
timeData = motionRawData[:, 0]

In [120]:
# Motion data
temp = motionRawData[:, 1]
motionData = signal.savgol_filter(temp, 151, 3)
motionData = np.interp(motionData, (motionData.min(), motionData.max()), (0, 10))
#from sklearn import preprocessing
#motionDataNorm = np.array(motionData[:, 1])
#motionDataNorm = preprocessing.normalize([motionDataNorm])[0]
#motionDataNorm

In [121]:
# Normalize FSR data
#FSRData = scale(np.array(FSR_IR[:, 0]))
FSRData = np.array(FSR_IR[:, 0])
FSRData = signal.savgol_filter(FSRData, 151, 3)
FSRData = signal.savgol_filter(FSRData, 151, 3)
FSRData = np.interp(FSRData, (FSRData.min(), FSRData.max()), (0, 10))
#FSRData = move(FSRData)
#FSRDataNorm = preprocessing.normalize([temp])[0]
#FSRDataNorm.shape

In [122]:
# Normalize IR data
#IRData = scale(np.array(FSR_IR[:, 1]))
IRData = np.array(FSR_IR[:, 1])
IRData = signal.savgol_filter(IRData, 151, 3)
IRData = signal.savgol_filter(IRData, 151, 3)
IRData = np.interp(IRData, (IRData.min(), IRData.max()), (0, 10))
#IRData = move(IRData)
#IRDataNorm = preprocessing.normalize([temp])[0]
#IRDataNorm.shape

In [123]:
#Scatter plot
import plotly as py
import plotly.graph_objs as go
from scipy import signal

# Create traces
trace0 = go.Scatter(
    x = timeData,
    y = motionData,
    mode = 'lines',
    name = 'Motion'
)
trace1 = go.Scatter(
    x = timeData,
    y = FSRData,
    mode = 'lines',
    name = 'FSR'
)
trace2 = go.Scatter(
    x = timeData,
    y = IRData,
    mode = 'lines',
    name = 'IR'
)

#data = [trace0, trace1, trace2]
data = [trace0, trace1, trace2]

# Plot and embed in ipython notebook!
py.offline.plot(data, filename='Scatter.html')

'file://C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\PostProcessing\\Python\\Scatter.html'

In [124]:
#Correlation coefficient
#np.corrcoef(motionData, FSRData), np.corrcoef(motionData, IRData), np.corrcoef(FSRData, IRData)

(array([[1.        , 0.93499634],
        [0.93499634, 1.        ]]), array([[1.        , 0.47640641],
        [0.47640641, 1.        ]]), array([[1.        , 0.54982098],
        [0.54982098, 1.        ]]))

In [131]:
data2 = [go.Bar(
            x=['Motion vs. FSR', 'Motion vs. IR', 'IR vs. FSR'],
            y=[np.corrcoef(motionData, FSRData)[0][1], np.corrcoef(motionData, IRData)[0][1], np.corrcoef(FSRData, IRData)[0][1]])]

layout = go.Layout(
    title='Correlation Coefficient of Sensors',
)

fig = go.Figure(data=data2, layout=layout)
py.offline.plot(fig, filename='Bar.html')

'file://C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\PostProcessing\\Python\\Bar.html'

In [92]:
#Joining time + motion
temp = np.vstack((timeData, motionData))

size1 = temp[0].size
size2 = temp.size

temp = np.reshape(temp, size2, order='F')

temp = temp.reshape((size1, 2))
temp1 = temp
temp1.shape

(1232, 2)

In [93]:
#joining FSR + IR
temp = np.vstack((FSRData, IRData))
size1 = temp[0].size
size2 = temp.size

temp = np.reshape(temp, size2, order='F')

temp = temp.reshape((size1, 2))
temp2 = temp
temp2.shape

(1232, 2)

In [94]:
#Joining TimeMotion + FSRIR
final = np.hstack((temp1, temp2))
final, final.shape

(array([[ 0.4       , 10.        ,  1.06412869,  0.        ],
        [ 0.4       ,  9.75413047,  1.2365006 ,  0.09717938],
        [ 0.44      ,  9.51199562,  1.40724152,  0.19346827],
        ...,
        [59.96      ,  5.08069565,  2.97424541,  3.10324491],
        [60.        ,  5.43052125,  3.10431292,  3.0810266 ],
        [60.        ,  5.79414065,  3.24077237,  3.0584794 ]]), (1232, 4))

In [96]:
#Saving to file
#np.savetxt(r"Data\out.txt", final, delimiter=",", fmt='%s')